# Triple Pendulum
Created by: Wesley Terrill <br/>
Revision notes:<br/>
(04/23/2019): Took sections from Double Pendulum code to adapt to triple pendulum

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from scipy import linalg as LA 

from scipy.integrate import odeint

from mpmath import *

#import functions from sympy
import sympy as sym
from sympy import *
#from sympy.mpmath import *

#from sympy.functions.elementary.trigonometric import sin, cos
from sympy.physics.vector import dynamicsymbols
from sympy import diff, Symbol
from sympy.printing.mathml import print_mathml
from sympy.abc import t
from sympy.utilities.lambdify import lambdify, implemented_function
from sympy import Function

from IPython.display import display, Math #for printing latex string

from scipy.integrate import solve_ivp

from matplotlib import animation, rc
from IPython.display import HTML, Image
from matplotlib import animation, rc



Find the expressions for position, velocity, and acceleration in 
terms of variables that specify the state of the device.

$\begin{align}
    x = horizontal \; position \; of \; pendulum \; mass   
\\
    y = vertical \; position \; of \; pendulum \; mass    
\\
    \theta = angle \; of \; pendulum \; (0 \; is \; vertically \;   
    downwards, \;
    counter-clockwise \; is \; positive)
\\
    L = length \; of  \; rod \; (constant)
\end{align}$   


Place the origin at the pivot of the upper pendulum. Take y increasing as upwards. Indicate upper pendulum by subscript 1, and lower by subscript 2. Begin by using simple geometry to write 
$x_1, \; y_1,\;x_2,\;y_2,\;x_3,\;y_3$ in terms of the angles $\theta_1,\; \theta_2,\; \theta_3$. 

Note: written below for double pendulum only. Code adjusted to symbollically solve.

In [ ]:
display(Image(url='https://chegg-html-solutions.s3.amazonaws.com/9780134361307/719688-6-58P-i1.png'))

$\begin{align}
  x_1 = L_1 \sin(\theta_1)
\\
  y_1 = -L_1 \cos(\theta_1)
\\
  x_2 = x_1 + L_2 \sin(\theta_2)
\\
  y_2 = y_1 - L_2 \cos(\theta_2)
\\
  x_3 = x_2 + L_3 \sin(\theta_3)
\\
  y_3 = y_2 - L_3 \cos(\theta_3)
\end{align}$

The velocity is the derivative with respect to time.

$
  \dot{x_1} = L_1 \dot{\theta_1} \cos(\theta_1)
  $
  
  $
  \dot{y_1} = L_1 \dot{\theta_1} \sin(\theta_1)
  $
  
  $
  \dot{x_2} = \dot{x_1} + L_2 \dot{\theta_2} \cos(\theta_2)
  $
  
  $
  \dot{y_2} = \dot{y_1} + L_2 \dot{\theta_2} \sin(\theta_2)
$

First, set up a list of symbols capable of performing requested tasks.

In [ ]:
# define dynamic symbols (the angles for any number of masses attached)
theta_1, theta_2, theta_3 = dynamicsymbols('theta_1, theta_2, theta_3')

# define dynamic symbols for substitution later
phi_1, phi_2, phi_3 = symbols('phi_1, phi_2, phi_3')

#create derivative of the angles
theta_1_dot = diff(theta_1, t)
theta_2_dot = diff(theta_2, t)
theta_3_dot = diff(theta_3, t)

#create derivative of the sub terms
phi_1_dot = symbols(r'\dot{\phi_1}')
phi_2_dot = symbols(r'\dot{\phi_2}')
phi_3_dot = symbols(r'\dot{\phi_3}')


### define constants ###
#gravity
g = symbols('g', integer=True, positive=True)

#lengths
L1 = symbols('L_1', integer=True, positive=True)
L2 = symbols('L_2', integer=True, positive=True)
L3 = symbols('L_3', integer=True, positive=True)

#masses
m1 = symbols('m_1', integer=True, positive=True)
m2 = symbols('m_2', integer=True, positive=True)
m3 = symbols('m_3', integer=True, positive=True)

In [ ]:
### define cartesian in terms of theta ###
# first mass at top pendulum
x1 = L1 * sin(theta_1)
y1 = -L1 * cos(theta_1)

# second mass
x2 = x1 + L2 * sin(theta_2)
y2 = y1 - L2 * cos(theta_2)

#third mass
x3 = x2 + L3 * sin(theta_3)
y3 = y2 - L3 * cos(theta_3)

### define cartesian velocities ###
#first mass
x1_dot = diff(x1, t)
y1_dot = diff(y1, t)

#second mass
x2_dot = diff(x2, t)
y2_dot = diff(y2, t)

#third mass
x3_dot = diff(x3, t)
y3_dot = diff(y3, t)

#debugging line, sub in variable for displayig test
#display(Math(latex(y3_dot)))

In [ ]:
class multi_pendulum():
    """
    multi_pendulum class implements the parameters to solve 
    Lagrangian for motion of system of multiple pendulums
    
    Parameters
    ----------
    q(t) : dynamic symbol array
        position of mass as a function of time in terms of angle
        to be solved for
        will be defined outside class
        
    q_dot(t) : dynamic symbol array
        angular velocity of mass as a function of time
        defined outside class
    
    Methods
    -------
    Lagrangian
        creates lagrangian given kinetic energy and potential energy
        in terms of q(t) and q_dot(t)
        Returns the Lagrangian
        
    Lagrange
        takes Lagrangian and uses to solve the Lagrange equations
        Returns two lists of each half of differential equations
    """
    
    def __init__(self, q, q_dot, p, p_dot):
        self.q = q
        self.q_dot = q_dot
        self.p = p
        self.p_dot = p_dot
        
    def Lagrangian(self, U, T):
        """
        Forms the Lagrangian of the pendulum system given Potential 
        and Kinetic energy of system
        Solves Lagranges equations symbolically
        Returns the Lagrange
        """
        
        self.Lagrangian = simplify(T - U)
        
        return self.Lagrangian
    
    def Lagrange(self):
        """
        takes Lagrangian and uses to solve the Lagrange equations
        Returns two lists of each half of differential equations
        """
        # intialize list of ODE for each term
        self.ODE = []
        
        # iterates for each angle variable
        for i in range(0, len(self.q)):
            #Derivative with respect to q
            dL_dq = diff(self.Lagrangian, self.q[i])
        
            #Derivative with respect to q_dot
            dL_dq_dot = simplify(diff(self.Lagrangian, self.q_dot[i]))
        
            #Time derivative
            dL_dq_dot_dt = diff(dL_dq_dot, t)
            
            #assemble as function
            lagrange = simplify(Eq(dL_dq,dL_dq_dot_dt))
        
            #loop substitutes variables earlier for ease of reading using vector U
            for j in range(len(self.q)):
                lagrange = lagrange.subs([(diff(self.q_dot[j],t), self.p_dot[j]), (self.q_dot[j], self.p[j]), \
                                           (self.q[j],rf'theta_{j+1}')])
            
            diffeq = solve(lagrange, self.p_dot[i])
            #self.ODE.append([diff(diff(self.q[i],t),t), diffeq])
            self.ODE.append(diffeq)
    
        return simplify(self.ODE)

In [ ]:
class triple_pendulum(multi_pendulum):
    """
    triple_pendulum is a subClass of multipendulum optimized for the 
    double pendulum problem
    
    Parameters
    ----------
    q(t) : dynamic symbol array
        position of mass as a function of time in terms of angle
        to be solved for
        will be defined outside class
        
    q_dot(t) : dynamic symbol array
        angular velocity of mass as a function of time
        defined outside class
        
    p(t) : dynamic symbol array
        equivalent substituent of q_dot(t)
        defined outside class
    
    p_dot(t) : dynamic symbol array
        equivalent substituent to second derivative of q(t)
        with respect to time
        
    L_i : numeric value
        numeric length of pendulum i 
        
    m_i : numeric value
        numeric mass of pendulum i
        
    grav : numeric value
        gravitational acceleration
        
    Methods
    -------
    substitute
        substitutes in the numeric variables for length, mass, and
        gravitational acceleration into the equations of motion.
        Also substitutes the dynamic variables for normal constant
        values.
        
    conversion
        converts the numerically substituted variables to numpy usable
        functions using lambdify.
        
    dU_dx_old
        Right side of the differential equation to be solved.
        U is a four-component vector with theta_1=U[0], phi_1=U[1],
        theta_2=U[2], phi_2=U[3], theta_2=U[4], phi_2=U[5]. 
        Thus this function should return [phi_1, phi_1', phi_2, phi_2', 
        phi_3, phi_3']
        Uses direct copy of code from phi_1', phi_2', and phi_3' in 
        place of lambdify generated functions
    """
    def __init__(self, q, q_dot, p, p_dot, theta, phi,
                 L_1=1., L_2=1., L_3=1., m_1=1., m_2=1., m_3=1.,
                 grav=1.):
        multi_pendulum.__init__(self, q, q_dot, p, p_dot)
        self.L_1 = L_1
        self.L_2 = L_2
        self.L_3 = L_3
        self.m_1 = m_1
        self.m_2 = m_2
        self.m_3 = m_3
        self.grav = grav
        
        self.p = p
        self.p_dot = p_dot
        
        self.theta = theta
        self.phi = phi
    
    def substitute(self):
        """
        solves the angular acceleration terms independently of each other
        acceleration components
        """
        ODE = self.Lagrange()
        y=[ODE[0][0],ODE[1][0],ODE[2][0]]
        
        self.z = []
        subbed=[]
        
        self.z = simplify(solve([Eq(y[0],self.p_dot[0]),
                   Eq(y[1],self.p_dot[1]),
                   Eq(y[2],self.p_dot[2])],
                 self.p_dot[0], self.p_dot[1], self.p_dot[2],
                 dict = False, set = False, implicit = False))
        
        return self.z
    
    # following function used for future updates
    def conversion(self): 
        """
        converts input to a lambdify generated function
        """
        # call substituted equations of motion
        numODE = []
        for i in range(len(self.z)):
            numODE.append(self.z[self.p_dot[i]])
        
        # initialize empty array of the converted function
        self.npfunc = []
        
        # loop through the equations of motion
        for i in range(len(numODE)):
            f = lambdify((self.theta[0], self.theta[1], self.theta[2],
                          self.phi[0], self.phi[1], self.phi[2],
                         'L_1', 'L_2', 'L_3',
                          'm_1', 'm_2', 'm_3',
                          'g'), numODE[i], np)
            self.npfunc.append(f)
            
        return self.npfunc
    
    # Define a function for the right side
    def dU_dx_old(self, t, U):
        """
        Right side of the differential equation to be solved.
        U is a four-component vector with theta_1=U[0], phi_1=U[1],
        theta_2=U[2], phi_2=U[3], theta_3=U[4], phi_3=U[5]. 
        Thus this function should return [phi_1, phi_1', phi_2, phi_2', phi_3, phi_3']
        """
        #self.npfunc = self.conversion()
        
        # create the first diff eq from lambdify generated equations
        phi_1_dot = self.npfunc[0](U[0], U[2], U[4], U[1], U[3], U[5],
                                   self.L_1, self.L_2, self.L_3,
                                   self.m_1, self.m_2, self.m_3,
                                   self.grav)
        
        # created the second diff eq from lambdify generated equations
        phi_2_dot = self.npfunc[1](U[0], U[2], U[4], U[1], U[3], U[5], 
                                   self.L_1, self.L_2, self.L_3,
                                   self.m_1, self.m_2, self.m_3,
                                   self.grav)
        
        # created the third diff eq from lambdify generated equations
        phi_3_dot = self.npfunc[2](U[0], U[2], U[4], U[1], U[3], U[5],
                                   self.L_1, self.L_2, self.L_3,
                                   self.m_1, self.m_2, self.m_3,
                                   self.grav)
        
        return [U[1], phi_1_dot, U[3], phi_2_dot, U[5], phi_3_dot] 
    
    def Solve_ODE_old(self, t_pts,
                      theta_01, phi_01,
                      theta_02, phi_02,
                      theta_03, phi_03):
        """
        solves the equations of motion using solve_ivp.
        Bring in array of time points, initial angles,
        and angular velocities.
        """
        U_0 = [theta_01, phi_01, theta_02, phi_02, theta_03, phi_03]
        solution = solve_ivp(self.dU_dx_old, (t_pts[0], t_pts[-1]),\
                             U_0, t_eval = t_pts, method='Radau',\
                             atol = 1.0e-7, rtol = 1.0e-7)
        
        theta_1, phi_1, theta_2, phi_2, theta_3, phi_3 = solution.y
        print(solution.message, solution.success)
        return theta_1, phi_1, theta_2, phi_2, theta_3, phi_3

In [ ]:
#Create lists of variables that are substituted
theta1 = symbols('theta_1')
theta2 = symbols('theta_2')
theta3 = symbols('theta_3')

theta = [theta1, theta2, theta3]
phi = [phi_1, phi_2, phi_3]
phi_dot = [phi_1_dot, phi_2_dot, phi_3_dot]

### double pendulum energy ###
# potential
U_trip = m1 * g * y1 + m2 * g * y2 + m3 * g * y3

# Kinetic
T_trip = simplify(m1/2 * (x1_dot**2 + y1_dot**2) +\
             m2/2 * (x2_dot**2 + y2_dot**2) +\
             m3/2 * (x3_dot**2 + y3_dot**2))

### initialize multi_pendulum class ###
# create lists of required variables
theta_t = [theta_1, theta_2, theta_3]
theta_dot = [theta_1_dot, theta_2_dot, theta_3_dot]
phi_t = [phi_1, phi_2, phi_3]
phi_dot_t = [phi_1_dot, phi_2_dot, phi_3_dot]
             
#Define the constants values
L_1 = 1.
L_2 = 1.
L_3 = 1.
m_1 = 1.
m_2 = 1.
m_3 = 1.
g = 1.

In [ ]:
# initialize class
triple_pen = triple_pendulum(q = theta_t, q_dot = theta_dot, p = phi_t, \
                             p_dot = phi_dot_t, theta = theta, phi = phi,\
                             L_1 = L_1, L_2 = L_2, L_3 = L_3,\
                             m_1 = m_1, m_2 = m_2, m_3 = m_3,\
                             grav = g)

lag = triple_pen.Lagrangian(U = U_trip, T = T_trip)

#view lagrange
display(Math(latex(lag)))

Not a fun Lagrangian to solve. Luckily, program should be able to do it easily.

In [ ]:
# solve lagrange equations
diffeq = triple_pen.Lagrange()
display(Math(latex((diffeq))))

The next step runs slowly as it solves the three equations motion for three new equations with the angular acceleration terms independent of the others. Solving the equations of angular acceleration to be independent of each other yields the following in order:

In [ ]:
numODE = triple_pen.substitute()
display(Math(latex(simplify(numODE))))


In [ ]:
# required to convert equations before being called to solve equations
triple_pen.conversion();

In [ ]:
#the time points
t_start = 0.
t_stop = 200.
delta_t = 0.1
t_pts = np.arange(t_start, t_stop + delta_t, delta_t)


In [ ]:
# Set initial conditions (angles in radians)
theta_01 = -np.pi/4.
theta_02 = -np.pi/2.
theta_03 = np.pi/1.1
phi_01 = 0.
phi_02 = 0.
phi_03 = 0.

theta1, phi1, theta2, phi2, theta3, phi3 = triple_pen.Solve_ODE_old(t_pts = t_pts, \
                                                      theta_01 = theta_01, \
                                                      phi_01 = phi_01, \
                                                      theta_02 = theta_02, \
                                                      phi_02 = phi_02,\
                                                      theta_03 = theta_03,\
                                                      phi_03 = phi_03)   
    
#print(len(t_pts))
#print(len(theta1))

In [ ]:
def plot(angles, velocity, t_pts, colors):
    """
    creates plots of angles versus time and phase-space plots
    """
    fig = plt.figure(figsize=(5,5), num='pendulum')
    ax = fig.add_subplot(2,1,1)
    ax2 = fig.add_subplot(2,1,2)
    
    for i in range(len(angles)):
        ax.plot(t_pts, angles[i], linestyle='dashed', color=colors[i],\
                label=rf'$\theta_{i+1}$')
        
        ax2.plot(angles[i], velocity[i], linestyle='dashed', color=colors[i],\
                label=rf'$\theta_{i+1}$')
        
    
    ax.legend()

    ax.set_xlabel('t')
    ax.set_ylabel(rf'$\theta(t)$')
    
    ax2.legend()

    ax2.set_ylabel(r'$\dot{\theta}(t)$')
    ax2.set_xlabel(rf'$\theta(t)$')
    
    fig.tight_layout()


In [ ]:
angles = [theta1, theta2, theta3]
velocity = [phi1, phi2, phi3]
colors = ['red', 'blue', 'green']
plot(angles = angles, velocity = velocity, t_pts = t_pts, colors = colors)

In [ ]:
def polar_to_cartesian(theta, length):
    """
    converts the angle value to cartesian coordinates with fixed lengths
    of pendulums
    """
    x = []
    y = []

    for i in range(len(theta)):
        
        x_now = length[i] * np.sin(theta[i])
        y_now = - length[i] * np.cos(theta[i])
        
        x.append(x_now)
        y.append(y_now)
    return x, y

In [ ]:
length = [L_1, L_2, L_3]
angle = [theta1, theta2, theta3]

x, y = polar_to_cartesian(theta = angle, length = length)

# first bob's position
x1 = x[0]
y1 = y[0]

# second bob's position
x2 = x1 + x[1]
y2 = y1 + y[1]

# third bob's position
x3 = x2 + x[2]
y3 = y2 + y[2]


In [ ]:
%%capture
        
fig_anim = plt.figure(figsize=(10,5), num = 'pendulum_anim')

#information for theta vs time animation
ax_anim = fig_anim.add_subplot(1,2,1)
ax_anim.set_xlabel('t')
ax_anim.set_ylabel(rf'$\theta$')
ax_anim.set_xlim(t_start, t_stop)

# lines of angle versus time plot
line_anim_angle, = ax_anim.plot(t_pts, theta1, color = 'blue', lw=2)
line_anim2_angle, = ax_anim.plot(t_pts, theta2, color = 'red', lw=2)
line_anim3_angle, = ax_anim.plot(t_pts, theta3, color = 'green', lw=2)

# points that trace angle versus time plot
pt_anim_angle, = ax_anim.plot(t_pts[0], theta1[0], 'o',
                              markersize=8, color='blue')
pt_anim2_angle, = ax_anim.plot(t_pts[0], theta2[0], 'o',
                               markersize=8, color='red')
pt_anim3_angle, = ax_anim.plot(t_pts[0], theta3[0], 'o',
                               markersize=8, color='green')
        
#information for pendulum animation
ax_anim_pend = fig_anim.add_subplot(1,2,2)
ax_anim_pend.set_xlim(-3.1, 3.1)
ax_anim_pend.set_ylim(-3.1, 3.1)
ax_anim_pend.set_aspect(1)
ax_anim_pend.axis('off')

### The rods that connect the pendulums ###
#rod from first bob to pivot
ln_anim_1, = ax_anim_pend.plot([0.,x1[0]], [0.,y1[0]],
                          '-', color = 'black')
#rod from first bob to second bob
ln_anim_2, = ax_anim_pend.plot([x1[0],x2[0]], [y1[0],y2[0]],
                          '-', color = 'black')  

#rod from second bob to third bob
ln_anim_3, = ax_anim_pend.plot([x2[0],x3[0]], [y2[0],y3[0]],
                          '-', color = 'black') 

#the pivot point
pivot, = ax_anim_pend.plot(0., 0., 'o', 
                           markersize = 8, color = 'black')

### The pendulum bob's ###
pt_anim_1, = ax_anim_pend.plot(x1[0], y1[0],'*',
                               markersize = 12, color = 'blue')

pt_anim_2, = ax_anim_pend.plot(x2[0], y2[0],'*',
                               markersize = 12, color='red')

pt_anim_3, = ax_anim_pend.plot(x3[0], y3[0],'*',
                               markersize = 12, color='green')

fig_anim.tight_layout()
        

In [ ]:
def animate_pendulum(i):
    """
    This is the function called by FuncAnimation to create each frame,
    numbered by i.  So each i corresponds to a point in the t_pts
    array, with index i.
    """
    i_skip = i
    
    #info for first plot
    t = t_pts[i_skip]
    angle_pt1 = theta1[i_skip]
    angle_pt2 = theta2[i_skip]
    angle_pt3 = theta3[i_skip]
        
    pt_anim_angle.set_data(t, angle_pt1)
    pt_anim2_angle.set_data(t, angle_pt2)
    pt_anim3_angle.set_data(t, angle_pt3)
    
    #info for second plot
    x_1 = x1[i_skip]
    y_1 = y1[i_skip]
    x_2 = x2[i_skip]
    y_2 = y2[i_skip]
    x_3 = x3[i_skip]
    y_3 = y3[i_skip]
    
    #rod connecting bobs
    ln_anim_1.set_data([0.0, x_1], [0.0, y_1])
    ln_anim_2.set_data([x_1, x_2], [y_1, y_2])
    ln_anim_3.set_data([x_2, x_3], [y_2, y_3])
    
    #pendulum bobs
    pt_anim_1.set_data(x_1, y_1)
    pt_anim_2.set_data(x_2, y_2)
    pt_anim_3.set_data(x_3, y_3)
        
    return(pt_anim_angle, pt_anim2_angle, pt_anim3_angle,
           ln_anim_1, ln_anim_2, ln_anim_3,
           pt_anim_1, pt_anim_2, pt_anim_3)

In [ ]:
frame_interval = 20.  # time between frames
frame_number = len(t_pts) # number of frames to include (index of t_pts)
anim = animation.FuncAnimation(fig_anim, 
                               animate_pendulum, 
                               init_func=None,
                               frames=frame_number, 
                               interval=frame_interval, 
                               blit=True,
                               repeat=False)

In [ ]:
plt.rcParams['animation.embed_limit'] = 90. 
HTML(anim.to_jshtml())


system is even more chaotic than the double pendulum